In [ ]:
import sys, os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
sys.path.append('..')
import avstack
import avapi
from copy import copy, deepcopy


import numpy as np
%load_ext autoreload
%autoreload 2

%matplotlib inline

data_dir = '../../data/CARLA/ego-lidar/'
CSM = avapi.carla.CarlaScenesManager(data_dir, verbose=False)

In [ ]:
CSD = CSM.get_scene_dataset_by_name(CSM.scenes[-1])
print(len(CSD.sensor_file_post[list(CSD.sensor_file_post.keys())[0]]['frame']), 'camera frames')

frame_ordering_cam = list(CSD.sensor_file_post['CAM_FRONT']['frame'].keys())
frame_ordering_ego = list(CSD.ego_files['frame'].keys())
frame_ordering = [x for x in frame_ordering_cam if x in frame_ordering_ego]

## Visualizations

In [ ]:
# Get scene dataset
frame = frame_ordering[30]
all_sensors = ['CAM_FRONT']
for cam in all_sensors:
    if 'DEPTH' in cam:
        img = CSD.get_depthimage(frame, cam)
    else:
        img = CSD.get_image(frame, cam)
    calib = CSD.get_calibration(frame, cam)
    ego = CSD.get_ego(frame)
    pc = CSD.get_lidar(frame, 'LIDAR_TOP')
    objects = CSD.get_objects(frame, 'LIDAR_TOP')
    
    # visualize the results
    avapi.visualize.snapshot.show_image_with_boxes(img, objects, inline=True)
    avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=objects, inline=True)

## Perception

#### 2D Perception

In [ ]:
M_carla = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(model='fasterrcnn', threshold=0.7, dataset='carla')
M_cityscapes = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(model='fasterrcnn', threshold=0.7, dataset='cityscapes')

In [ ]:
cam = 'CAM_FRONT'
frame = frame_ordering[30]
img = CSD.get_image(frame, sensor=cam)
dets_carla = M_carla(img)
dets_cityscapes = M_cityscapes(img)

In [ ]:
avapi.visualize.snapshot.show_image_with_boxes(img, dets_carla, inline=True)
avapi.visualize.snapshot.show_image_with_boxes(img, dets_cityscapes, inline=True)

#### 3D Perception

In [ ]:
# M3D = avstack.modules.perception.object3d.MMDetObjectDetector3D(
#     model='pointpillars', dataset='carla', threshold=0.3, gpu=1)  # NOTE: we haven't trained a carla model yet...